In [6]:
import numpy as np
import pyvista as pv
import gmsh
import pandas as pd
from collections import deque, defaultdict
import scipy.io
from scipy.spatial import ConvexHull
import open3d as o3d
import os

In [7]:
def connectivityExtractor(name):
    file_path = 'Networks/Network_Vessels_' + name +'.mat'
    matlab_data = scipy.io.loadmat(file_path)
    # Extract the 'connectivity' field from the 'Data' structured array
    data_structure = matlab_data['Data']
    connectivity_raw = data_structure['connectivity'][0, 0]  # Access the data (adjust indexing if needed)
    # Reshape or ensure it's a proper 2D array (if required)
    connectivity_data = connectivity_raw.squeeze()
    # Create a DataFrame from the connectivity data
    connectivity_df = pd.DataFrame(connectivity_data, columns=['Parent', 'Daughter1', 'Daughter2', 'Daughter3'])
    connectivity_df.replace(0, np.nan, inplace=True) #ensure all nonexistent vessels have NaN
    connectivity_df.at[0,'Parent']=0 #make sure first vessel is 0 (purposefully removed in last step for ease)
    # Save the DataFrame to inspect it
    return connectivity_df

def nodesExtractor(name): #extracts nodes and their corresponding information
    file_path = 'Networks/Network_Vessels_' + name +'.mat'
    matlab_data = scipy.io.loadmat(file_path)
    # Extract the 'connectivity' field from the 'Data' structured array
    data_structure = matlab_data['nodesC2']
    # Reshape or ensure it's a proper 2D array (if required)
    nodes_data = data_structure.squeeze()
    # Create a DataFrame from the connectivity data
    nodes_df = pd.DataFrame(nodes_data, columns=['NodeID', 'X', 'Y', 'Z', 'Degree'])
    # Save the DataFrame to inspect it
    return nodes_df

def edgesExtractor(name): #extracts segments to create a dataframe of from and to nodes
    file_path = 'Networks/Network_Vessels_' + name +'.mat'
    matlab_data = scipy.io.loadmat(file_path)
    # Extract the 'segments' field
    data_structure = matlab_data['segments']
    # Reshape or ensure it's a proper 2D array (if required)
    edges_data = data_structure.squeeze()
    # Create a DataFrame from the connectivity data
    edge_df = pd.DataFrame(edges_data, columns=['ID', 'From', 'To'])
    # Save the DataFrame to inspect it
    return edge_df
    
def findInputVessel(segments,fromnode,to):
    vessel = segments[((segments['From'] == fromnode)&(segments['To']==to))|((segments['From'] == to)&(segments['To']==fromnode))]
    return int(vessel['ID'])

def mapIDExtractor(name):
    file_path = 'Networks/Network_Vessels_' + name +'.mat'
    matlab_data = scipy.io.loadmat(file_path)
    # Extract the 'mapID' field from the 'Data' structured array
    data_structure = matlab_data['Data']
    map_raw = data_structure['mapIDs'][0, 0]  # Access the data (adjust indexing if needed)
    # Reshape or ensure it's a proper 2D array (if required)
    map_data = map_raw.squeeze()
    # Create a DataFrame from the connectivity data
    map_df = pd.DataFrame(map_data, columns=['New', 'Old'])
    # Save the DataFrame to inspect it
    return map_df

def lobeExtractor(name, vesID):
    data = connectivityExtractor(name)
    
    tree = defaultdict(list)
    for _,row in data.iterrows():
        parent = row['Parent']
        for daughter_col in ['Daughter1','Daughter2','Daughter3']:
            daughter = row[daughter_col]
            if pd.notna(daughter):
                tree[parent].append(daughter)

    visited = set()
    queue = deque([vesID])

    while queue:
        current = queue.popleft()
        if current not in visited:
            visited.add(current)
            queue.extend(tree.get(current,[]))
    
    downstream_df = data[data['Parent'].isin(visited)]
    return downstream_df

def term_nodes_loc(name,lobe_nodes):
    nodes = nodesExtractor(name)
    lobe = nodes[nodes['NodeID'].isin(lobe_nodes)]
    termNodes = lobe[(lobe['Degree'] == 1)]
    return termNodes[['X','Y','Z']]

def node_loc(name,lobe_nodes):
    nodes = nodesExtractor(name)
    lobe = nodes[nodes['NodeID'].isin(lobe_nodes)]
    return lobe[['X','Y','Z']]

def lobeTermLoc(name,fromnode,tonode):
    segments = edgesExtractor(name)
    maps = mapIDExtractor(name)
    vesID = findInputVessel(segments,fromnode,tonode)
    newID = int(maps[maps['Old']==vesID]['New'])
    lobe_ves = lobeExtractor(name,newID)
    new_lobe_ves_ID = lobe_ves['Parent'].to_numpy()
    oldID = maps[maps['New'].isin(new_lobe_ves_ID)]['Old'].to_numpy()
    fromnodes = segments[segments['ID'].isin(oldID)]['From'].to_numpy()
    tonodes = segments[segments['ID'].isin(oldID)]['To'].to_numpy()
    lobe_nodes = np.unique(np.concatenate((fromnodes,tonodes))).astype(int)
    lobe_node_loc = node_loc(name,lobe_nodes)/1000
    #term_nodes = term_nodes_loc(name,lobe_nodes)
    return lobe_node_loc

def compute_mesh_volume():
    types, elem_tags, elem_nodes = gmsh.model.mesh.getElements(dim=3)
    total_volume = 0.0

    for etype, tags, nodes in zip(types,elem_tags,elem_nodes):
        if etype !=4:
            continue
        nodes = np.array(nodes).reshape(-1,4)
        node_tags,node_coords,_ = gmsh.model.mesh.getNodes()
        node_coords = np.array(node_coords).reshape(-1,3)
        node_map = dict(zip(node_tags,node_coords))

        for tet in nodes:
            p0 = node_map[tet[0]]
            p1 = node_map[tet[1]]
            p2 = node_map[tet[2]]
            p3 = node_map[tet[3]]
        
        vol = np.abs(np.dot((p1-p0),np.cross((p2-p0),(p3-p0))))/6
        total_volume += vol
        print(f"Total mesh volume: {total_volume}")
        return total_volume

In [3]:
name = 'm3p4_060407'
left_lobe = lobeTermLoc(name,939,955).to_numpy()
np.save('left_lobe_loc',left_lobe)

In [ ]:
name = 'm3p4_060407'
left_lobe = lobeTermLoc(name,939,955).to_numpy()
sup1loc = lobeTermLoc(name,991,989).to_numpy()
middleloc = lobeTermLoc(name,1033,1027).to_numpy()

sup2loc = lobeTermLoc(name,1056,1035).to_numpy()
suploc = np.concatenate((sup1loc,sup2loc))
inferiorloc = lobeTermLoc(name,1056,963).to_numpy()
post_cavalloc = lobeTermLoc(name,1056,1087).to_numpy()

In [ ]:
name = 'm3p4_060407'
left_lobe = lobeTermLoc(name, 939, 955).to_numpy()
points = np.unique(left_lobe, axis=0)

# Build a convex hull
hull = ConvexHull(points)
volume = hull.volume
# Initialize Gmsh
gmsh.initialize()
gmsh.model.add("volume_from_convex_hull")

# Create discrete surface entity
gmsh.model.addDiscreteEntity(2, 1)  # dim=2, tag=1

# Add nodes from your point cloud
node_tags = np.arange(1, len(points) + 1)
gmsh.model.mesh.addNodes(2, 1, node_tags.tolist(), points.flatten().tolist())

# Add triangular faces from the convex hull
triangles = hull.simplices + 1  # Gmsh uses 1-based indexing
gmsh.model.mesh.addElements(2, 1, [2], [np.arange(1, len(triangles) + 1)], [triangles.flatten()])

# Convert mesh surface into geometry
gmsh.model.mesh.classifySurfaces(30 * np.pi / 180., True, False, 180 * np.pi / 180.)
gmsh.model.mesh.createGeometry()
gmsh.model.geo.synchronize()

# Build volume from surface shell
surfaces = gmsh.model.getEntities(2)
surface_tags = [s[1] for s in surfaces]
loop = gmsh.model.geo.addSurfaceLoop(surface_tags)
volume = gmsh.model.geo.addVolume([loop])
gmsh.model.geo.synchronize()
gmsh.write("./Mesh/left_lobe_surface.stl")
# Tag volume for export
gmsh.model.addPhysicalGroup(3, [volume], name="EnclosedVolume")

# Generate 3D tetrahedral mesh
gmsh.option.setNumber('Mesh.CharacteristicLengthMin',.1) #min mesh distance
gmsh.option.setNumber('Mesh.CharacteristicLengthMax',.5)#max mesh distance
#ensures mesh doesn't overoptimize and get stuck in a loop
gmsh.option.setNumber('Mesh.MeshOnlyVisible',1)
gmsh.option.setNumber('Mesh.HighOrderOptimize',0)
gmsh.option.setNumber('Mesh.RecombineAll',0)
gmsh.option.setNumber('Mesh.Smoothing',0)
gmsh.option.setNumber('Mesh.Optimize',0)
gmsh.model.mesh.generate(3)
#visualizes mesh before writing to unv
gmsh.fltk.run()
#volume = compute_mesh_volume()
# Export as UNV
gmsh.write("./Mesh/left_lobe_mesh.unv")

gmsh.finalize()
print(volume)

Info    : Classifying surfaces (angle: 30)...
Info    : Found 62 model surfaces
Info    : Found 99 model curves
Info    : Done classifying surfaces (Wall 0.00144442s, CPU 0.001244s)
Info    : Creating geometry of discrete curves...
Info    : Done creating geometry of discrete curves (Wall 3.1417e-05s, CPU 3.7e-05s)
Info    : Creating geometry of discrete surfaces...
Info    : [ 10%] Discrete surface 2 is planar, simplifying parametrization                                        
Info    : [ 10%] Discrete surface 3 is planar, simplifying parametrization
Info    : [ 10%] Discrete surface 7 is planar, simplifying parametrization
Info    : [ 10%] Discrete surface 8 is planar, simplifying parametrization
Info    : [ 20%] Discrete surface 10 is planar, simplifying parametrization                                       
Info    : [ 20%] Discrete surface 11 is planar, simplifying parametrization
Info    : [ 20%] Discrete surface 12 is planar, simplifying parametrization
Info    : [ 20%] Discret

: 

In [3]:
name = 'm3p4_060407'
points = lobeTermLoc(name,939,955).to_numpy()
# Convert to Open3D point 
cloudpcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(points)

: 

In [3]:
name = 'm3p4_060407'
left_lobe = lobeTermLoc(name,939,955).to_numpy()
# Convert to Open3D point 
cloudpcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(left_lobe)
pcd.estimate_normals()
# Alpha shape surface reconstruction
alpha = 5 # Tune as needed
mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_alpha_shape(pcd, alpha)
mesh.compute_vertex_normals()
# Save STL
stl_file = "left_lobe_mesh.stl"
o3d.io.write_triangle_mesh(stl_file, mesh)

: 

In [ ]:
# Use Gmsh to load STL and compute volume
gmsh.initialize()
gmsh.option.setNumber("General.Terminal", 1)
gmsh.merge(stl_file)
# Classify surfaces and make volume
angle = 40
gmsh.model.mesh.classifySurfaces(angle * np.pi / 180, includeBoundary=True)
gmsh.model.mesh.createGeometry()
surfaces = gmsh.model.getEntities(dim=2)
loop = gmsh.model.geo.addSurfaceLoop([s[1] for s in surfaces])
vol = gmsh.model.geo.addVolume([loop])
gmsh.model.geo.synchronize()
# Mesh volume
gmsh.model.mesh.generate(3)
# Get nodes and elements
types, elem_tags, elem_nodes = gmsh.model.mesh.getElements(dim=3)
node_tags, node_coords_flat, _ = gmsh.model.mesh.getNodes()
node_coords = np.array(node_coords_flat).reshape(-1, 3)
tag_to_index = {tag: i for i, tag in enumerate(node_tags)}
# Compute volume from tetrahedra
total_volume = 0.0
for etype, tags, nodes in zip(types, elem_tags, elem_nodes):
    if etype != 4: continue
    tets = np.array(nodes).reshape(-1, 4)
    for tet in tets:
        p0 = node_coords[tag_to_index[tet[0]]]
        p1 = node_coords[tag_to_index[tet[1]]]
        p2 = node_coords[tag_to_index[tet[2]]]
        p3 = node_coords[tag_to_index[tet[3]]]
        vol = np.abs(np.dot((p1 - p0), np.cross(p2 - p0, p3 - p0))) / 6.0
        total_volume += vol
    
gmsh.fltk.run()

# Export as UNV
gmsh.write("./Mesh/left_lobe_mesh.unv")
gmsh.finalize()
print(f"Computed Volume: {total_volume:.4f} cubic units")